<a href="https://colab.research.google.com/github/amirhossini/Pyomo-Educational-Notebooks/blob/main/Pyomo3_MILP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pyomo Examples

__Notebook:__ Mixed Integer Linear Programming (MILP) 

__Questions:__ amir.hossini@queensu.ca

_Libraries_

In [1]:
!pip install pyomo
!apt-get install -y -qq glpk-utils
import pyomo.environ as pyomo
!pip install xlsxwriter
import xlsxwriter
import os

     |████████████████████████████████| 9.6 MB 3.9 MB/s 
     |████████████████████████████████| 49 kB 4.5 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155455 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up 

# Section 1: Example 1 - Dorian Auto Manufacturing

In [6]:
# Model definition
model = pyomo.ConcreteModel();

# Set definition
model.a = pyomo.Set(initialize = ['compact','midsize','large']);

# Defining big M parameter to be used as upper bound for variable x
model.M = pyomo.Param(model.a,initialize = {'compact':2000,'midsize':2000,'large':1200});

# Function that returns bounds on variable x
def bounds_for_x(model,l):
  return (0,model.M[l])
# Here, l is just a placeholder for the set a

# Variable declaration
model.x = pyomo.Var(model.a,domain = pyomo.Integers,bounds = bounds_for_x);
# Note: bounds for x: lower bound = 0, upper bound depends on big M value - define M as parameter first!

model.y = pyomo.Var(model.a,domain=pyomo.Binary);

# Parameter declaration
model.phi = pyomo.Param(model.a,initialize = {'compact':2000,'midsize':3000,'large':4000});

model.mu = pyomo.Param(model.a,initialize = {'compact':1000,'midsize':1000,'large':1000});

model.alpha = pyomo.Param(model.a,within = pyomo.Any,initialize = {'compact':{'steel':1.5,'labor':30},
                                                                   'midsize':{'steel':3,'labor':25},
                                                                   'large':{'steel':5,'labor':40}});

model.sigma = pyomo.Param(initialize = 6000);

model.gamma = pyomo.Param(initialize = 60000);

# Constraint definition
def rule1(model,a):
  return model.x[a] <= model.M[a]*model.y[a]
model.eq1 = pyomo.Constraint(model.a,rule=rule1,doc = 'Part 1 for the either-or constraint equivalent for each car');

def rule2(model,a):
  return model.mu[a] - model.x[a] <= model.M[a]*(1-model.y[a])
model.eq2 = pyomo.Constraint(model.a,rule=rule2,doc = 'Part 2 for the either-or constraint equivalent for each car');

def rule3(model):
  return sum(model.alpha[a]['steel']*model.x[a] for a in model.a) <= model.sigma
model.eq3 = pyomo.Constraint(rule=rule3,doc = 'Steel constraint');

def rule4(model):
  return sum(model.alpha[a]['labor']*model.x[a] for a in model.a) <= model.gamma
model.eq4 = pyomo.Constraint(rule=rule4,doc = 'Labor constraint');

# Objective function definition
model.obj = pyomo.Objective(expr = sum(model.phi[a]*model.x[a] for a in model.a),sense=pyomo.maximize);
# Note: objective here is to maximize profit from manufacturing cars!

# Solver options
# Remember to add PATH for solver executable when running on Google Colab
results = pyomo.SolverFactory('glpk',executable='/usr/bin/glpsol').solve(model);

# Printing results
results.write()
print("\nRESULTS:");
print("\nTotal profit from manufacturing:",model.obj());
for a in model.a:
  print("\nNumber of cars of type ",a, "manufactured = ",model.x[a]());
for a in model.a:
  print("\nAre cars of type ",a,"manufactured? [1: yes, 0: no]",model.y[a]());

# Model listing - useful for debugging purposes!
model.pprint()

# Exporting solution as a text file - using implicit command
model.solutions.store_to(results)
results.write(filename = 'Pyomo_MILP_example_1_output_results.txt')

# Need to point the Colab notebook to the proper Drive location - move to prerequisites!

# Exporting solution as a text file - explicit user commands
f = open('Pyomo_MILP_example_1_output_results_user.txt','w');
f.write("\nTotal profit from manufacturing = %d" % model.obj());
for a in model.a:
  f.write("\nNumber of cars of type %s manufactured = %d" % (a,model.x[a]()));
for a in model.a:
  f.write("\nAre cars of type %s manufactured? (1:Yes, 0:No) = %d" % (a,model.y[a]()));
f.close()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 6000000.0
  Upper bound: 6000000.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 7
  Number of nonzeros: 19
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 5
      Number of created subproblems: 5
  Error rc: 0
  Time: 0.017372846603393555
# ----------------------------------------------------------
#   Solution Information
# ---------------------------

# Section 2: Example 2 - Power generation problem

In [ ]:
model = pyomo.ConcreteModel();

# Set definition
model.t = pyomo.Set(initialize = ['12am-6am','6am-9am','9am-3pm','3pm-6pm','6pm-12am']);
model.g = pyomo.Set(initialize = ['G1','G2','G3']);

# Variable declaration
model.x = pyomo.Var(model.g,model.t,domain = pyomo.NonNegativeReals);
model.s = pyomo.Var(model.g,model.t,domain = pyomo.NonNegativeIntegers);


# Parameter declaration
model.delta = pyomo.Param(model.t,initialize = {'12am-6am':15000,'6am-9am':30000,'9am-3pm':25000,'3pm-6pm':40000,'6pm-12am':27000});
model.theta = pyomo.Param(model.t,initialize = {'12am-6am':6,'6am-9am':3,'9am-3pm':6,'3pm-6pm':3,'6pm-12am':6});

model.alpha = pyomo.Param(model.g,within = pyomo.Any,initialize = 
                          {'G1':{'min_pow':850,'max_pow':2000,'min_cost':1000,'inc_cost':2.0,'start':2000,'gen_lim':12},
                           'G2':{'min_pow':1250,'max_pow':1750,'min_cost':2600,'inc_cost':1.3,'start':1000,'gen_lim':10},
                           'G3':{'min_pow':1500,'max_pow':4000,'min_cost':3000,'inc_cost':3.0,'start':500,'gen_lim':5}                              
                          });

def nbounds(model,g,t):
  return (0,model.alpha[g]['gen_lim'])
model.n = pyomo.Var(model.g,model.t,domain = pyomo.NonNegativeIntegers,bounds = nbounds);                         

def epsilon_rule(model,g,t):
  return model.theta[t]*model.alpha[g]['min_cost']
model.epsilon = pyomo.Param(model.g,model.t,initialize = epsilon_rule);

def phi_rule(model,g,t):
  return model.theta[t]*model.alpha[g]['inc_cost']
model.phi = pyomo.Param(model.g,model.t,initialize = phi_rule);

# Constraint declaration
def rule1(model,t):
  return sum(model.x[g,t] for g in model.g) >= model.delta[t]
model.eq1 = pyomo.Constraint(model.t,rule=rule1,doc = 'Power demand satisfaction');

def rule2(model,t):
  return sum(model.alpha[g]['max_pow']*model.n[g,t] for g in model.g) >= 1.15*model.delta[t]
model.eq2 = pyomo.Constraint(model.t,rule=rule2,doc = 'Spinning reserve requirement');

# Defining a new set tnew as a parameter to help with sequencing of time periods
model.tnew = pyomo.Param(model.t,initialize = {'12am-6am':5,'6am-9am':1,'9am-3pm':2,'3pm-6pm':3,'6pm-12am':4});

def rule3(model,g,t):
  ss = model.tnew[t];
  ss1 = model.t[ss];
  return model.s[g,t] >= model.n[g,t] - model.n[g,ss1]
model.eq3 = pyomo.Constraint(model.g,model.t,rule=rule3,doc = 'Start-up definition');

def rule4(model,g,t):
  return model.x[g,t] >= model.alpha[g]['min_pow']*model.n[g,t]
model.eq4 = pyomo.Constraint(model.g,model.t,rule=rule4,doc = 'Minimum generation levels');

def rule5(model,g,t):
  return model.x[g,t] <= model.alpha[g]['max_pow']*model.n[g,t]
model.eq5 = pyomo.Constraint(model.g,model.t,rule=rule5,doc = 'Maximum generation levels');

# Objective function definition
def obj_rule(model):
  term1 = sum(sum(model.epsilon[g,t]*model.n[g,t] for g in model.g) for t in model.t);
  term2 = sum(sum(model.alpha[g]['start']*model.s[g,t] for g in model.g) for t in model.t);
  term3 = sum(sum(model.phi[g,t]*(model.x[g,t]-model.alpha[g]['min_pow']*model.n[g,t]) for g in model.g) for t in model.t);
  return term1 + term2 + term3

model.obj = pyomo.Objective(rule=obj_rule,sense=pyomo.minimize);

# Solver options
#results = pyomo.SolverFactory('glpk',executable='/usr/bin/glpsol').solve(model);

# PLEASE REPLACE THE **** WITH YOUR EMAIL ID: for example, youremailid@something.com
os.environ['NEOS_EMAIL'] = '****';
solver_manager = pyomo.SolverManagerFactory('neos');
'''
['bonmin', 'cbc', 'conopt', 'couenne', 'cplex', 'filmint', 
'filter', 'ipopt', 'knitro', 'l-bfgs-b', 'lancelot', 'lgo', 'loqo', 'minlp', 'minos', 'minto', 'mosek', 'ooqp', 'path', 'raposa', 'snopt']
'''
results = solver_manager.solve(model,opt='cplex');

results.write()
print("\nRESULTS:");
print("\nTotal cost of power generation = ",model.obj());
for g in model.g:
  for t in model.t:
    print("\nOutput of generator",g,"during time period",t," = ",model.x[g,t]());
print("\n");
for g in model.g:
  for t in model.t:
    print("\nNumber of generators of type",g,"in use during time period",t," = ",model.n[g,t]());
print("\n");
for g in model.g:
  for t in model.t:
    print("\nNumber of generators of type",g,"started up during time period",t," = ",model.s[g,t]());

# Exporting solution as a .json file - implicit command
model.solutions.store_to(results);
results.write(filename='Pyomo_MILP_example_2_output_results.json');    

# Exporting solution as a .json file - explicit command
f = open('Pyomo_MILP_example_2_output_results_user.json','w');
f.write("\nTotal cost of power generation = %d" % (model.obj()));
for g in model.g:
  for t in model.t:
    f.write("\nOutput of generator %s during time period %s = %d" % (g,t,model.x[g,t]()));
f.write("\n");
for g in model.g:
  for t in model.t:
    f.write("\nNumber of generators of type %s in use during time period %s = %d" % (g,t,model.n[g,t]()))
f.write("\n");
for g in model.g:
  for t in model.t:
    f.write("\nNumber of generators of type %s started up during time period %s = %d" % (g,t,model.s[g,t]()));
f.close()



# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 55
  Number of variables: 45
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: CPLEX 20.1.0.0\x3a optimal integer solution; objective 988539.9999999999; 31 MIP simplex iterations; 0 branch-and-bound nodes
  Termination condition: optimal
  Id: 2
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions:

# Section 3: Example 3 - Flow Shop Scheduling

In [ ]:
model = pyomo.ConcreteModel();

# Set definition
model.i = pyomo.RangeSet(6);
model.m = pyomo.Set(initialize = ['bending','soldering','assembly']);

# Creating an alias set for i - used for ranking of the jobs in i
model.k = pyomo.Set(initialize = model.i);

# Variable declaration
model.R = pyomo.Var(model.i,model.k,domain = pyomo.Binary);
model.S = pyomo.Var(model.m,model.k,domain = pyomo.NonNegativeReals);
model.C = pyomo.Var(model.m,model.k,domain = pyomo.NonNegativeReals);

model.T = pyomo.Var();

model.tau = pyomo.Param(model.m,within=pyomo.Any,initialize = 
                        {'bending':{1:3,2:6,3:3,4:5,5:5,6:7},
                         'soldering':{1:5,2:4,3:2,4:4,5:4,6:5},
                         'assembly':{1:5,2:2,3:4,4:6,5:3,6:6}                            
                        });

# Constraint declaration
def rule1(model,k):
  return sum(model.R[i,k] for i in model.i) == 1
model.eq1 = pyomo.Constraint(model.k,rule=rule1,doc = 'Every position gets a job');

def rule2(model,i):
  return sum(model.R[i,k] for k in model.k) == 1
model.eq2 = pyomo.Constraint(model.i,rule=rule2,doc = 'Every job is assigned a rank');

def rule3(model,m,k):
  if k <= 5:
    return model.S[m,k+1] >= model.C[m,k]
  return pyomo.Constraint.Skip
model.eq3 = pyomo.Constraint(model.m,model.k,rule=rule3,doc = 'Relations b/w the end of job ranked k on machine m and start of job ranked k+1 on machine m');

model.mnew = pyomo.Param(model.m,initialize = {'bending':1,'soldering':2,'assembly':3});

def rule4(model,m,k):
  if model.mnew[m] <= 2:
    ss = model.mnew[m];
    mm = model.m[ss+1];
    return model.S[mm,k] >= model.C[m,k]
  return pyomo.Constraint.Skip
model.eq4 = pyomo.Constraint(model.m,model.k,rule=rule4,doc = 'Relations b/w the end of job ranked k on machine m and start of job ranked k on machine m+1');

def rule5(model,m,k):
  return model.C[m,k] == model.S[m,k] + sum(model.tau[m][i]*model.R[i,k] for i in model.i)
model.eq5 = pyomo.Constraint(model.m,model.k,rule=rule5,doc = 'Calculation of completion time based on start and processing times');

def rule6(model):
  return model.T >= model.C['assembly',6]
model.eq6 = pyomo.Constraint(rule=rule6,doc = 'Completion time of last job on last machine');

# Objective function definition
model.obj = pyomo.Objective(expr = model.T,sense = pyomo.minimize);

results = pyomo.SolverFactory('glpk',executable = '/usr/bin/glpsol').solve(model);

# Printing results
results.write();

print("\nRESULTS:");

print("\nTotal processing time (in min)",model.obj());
for i in model.i:
  for k in model.k:
    print("\nItem",i,"has position",k,"=",model.R[i,k]());
print("\n");
for m in model.m:
  for k in model.k:
    print("\nStart time on machine",m,"for job ranked",k,"=",model.C[m,k]());
print("\n");
for m in model.m:
  for k in model.k:
    print("\nCompletion time on machine",m,"for job ranked",k,"=",model.C[m,k]());
print("\n")

# Exporting solution as a .xlsx file - implicit command
model.solutions.store_to(results)
results.write(filename='Pyomo_MILP_example_3_output_results.xlsx');

# Exporting solution as a .xlsx file - explicit command
# Creating workbook
wb = xlsxwriter.Workbook("Pyomo_MILP_example_3_output_results_user.xlsx");

# Creating sheet
sheet = wb.add_worksheet();

# Writing headers for R[i,k]
row_headers_R = model.i.data();
col_headers_R = model.k.data();

# Note: row 1 or column 1 are identified as 0, row 2 or column 2 identified as 1, and so on...
for item in range(len(row_headers_R)):
  # Note: item + 1 : start from row 2 onwards (keep row 1 for column headers)
  sheet.write(item+1,0,"i%d" %(item+1))
for item in range(len(col_headers_R)):
  # Note: item + 1 : start from column 2 onwards (keep column 1 for row headers)
  sheet.write(0,item+1,"k%d" %(item+1))

# Writing R[i,k] into required cell range
for a in range(len(row_headers_R)):
  for b in range(len(col_headers_R)):
    # Extracting relevant set indices
    # Note: the +1 is due to a and b starting from 0 onwards, while i and k are RangeSets
    aa = model.i[a+1]; bb = model.k[b+1];
    sheet.write(a+1,b+1,"%d" % model.R[aa,bb].value);

# Writing headers for S[m,k]
row_headers_S = model.m.data();
col_headers_S = model.k.data();

for item in range(len(row_headers_S)):
  # Note: item + 1 : start from row 2 onwards, 9 : write to column 10 or 'J'
  # Note: %s because set m contains strings and not floats
  sheet.write(item+1,9,"%s" % model.m.data()[item]);
for item in range(len(col_headers_S)):
  # Note: 0 : write to row 2, item + 1 + 9 : start from column 10 or 'J'
  sheet.write(0,item+1+9,"k%d" % (item+1));

# Writing S[m,k] into required cell range
for a in range(len(row_headers_S)):
  for b in range(len(col_headers_S)):
    aa = model.m[a+1]; bb = model.k[b+1];
    sheet.write(a+1,b+1+9,"%d" % model.S[aa,bb].value);

# Writing headers for C[m,k]
row_headers_C = model.m.data();
col_headers_C = model.k.data();

for item in range(len(row_headers_C)):
  sheet.write(item+1+9,0,"%s" %model.m.data()[item]);
for item in range(len(col_headers_C)):
  sheet.write(9,item+1,"k%d" %(item+1));

# Writing C[m,k] into required cell range
for a in range(len(row_headers_C)):
  for b in range(len(col_headers_C)):
    aa = model.m[a+1]; bb = model.k[b+1];
    sheet.write(a+1+9,b+1,"%d" % model.C[aa,bb].value);

# Writing total processing time to specific cell
sheet.write("A15","Total processing time T = ");
sheet.write("B15",model.obj());

wb.close()



# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 35.0
  Upper bound: 35.0
  Number of objectives: 1
  Number of constraints: 59
  Number of variables: 74
  Number of nonzeros: 273
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 15
      Number of created subproblems: 15
  Error rc: 0
  Time: 0.029819488525390625
# ----------------------------------------------------------
#   Solution Information
# --------------------------------